In [1]:
suppressPackageStartupMessages(require(tidyverse))
suppressPackageStartupMessages(require(data.table))


In [2]:
phe_names <- fread('../common/canonical_trait_names.txt', head=T)

In [3]:
read_array_df <- function(){
    array_df <- fread('../meta_flipfix/jobs.lst', header=F)
    colnames(array_df) <- c('array')    

    array_df %>%
    mutate(
        annotation = str_replace(str_replace(basename(array), '^METAANALYSIS_', ''), '_1.tbl$', ''),
        array = str_replace(str_replace(array, '/meta/', '/meta_flipfixed/'), '.tbl$', '.tbl.gz'),
        annotation = str_replace(annotation, 'Apolipoprotein_B_adjust_statins', 'Apolipoprotein_B_adjstatins'),
        annotation = str_replace(annotation, 'LDL_direct_adjust_statins', 'LDL_direct_adjstatins'),
        annotation = str_replace(annotation, 'Cholesterol_adjust_statins', 'Cholesterol_adjstatins')        
    )
}

array_df <- read_array_df()

In [4]:
imp_df <- fread('imp_hits.lst', head=F) %>% 
rename('imp'= 'V1') %>%
mutate(
    annotation = str_replace_all(basename(imp), '^GLOBAL_|.1cm.hits$', '')
)

In [5]:
all_df <- phe_names %>%
left_join(
    array_df, by='annotation'
) %>%
left_join(
    imp_df, by='annotation'
) 

In [9]:
all_df %>%
# drop_na(c('array', 'imp')) %>% 
fwrite('cascade.input.files.tsv', sep='\t')


In [10]:
all_df %>%
filter(is.na(array) | is.na(imp))

name,annotation,array,imp
<chr>,<chr>,<chr>,<chr>
AST to ALT ratio,AST_ALT_ratio,/oak/stanford/groups/mrivas/projects/biomarkers_rivas/meta_flipfixed/METAANALYSIS_AST_ALT_ratio_1.tbl.gz,NA


In [11]:
all_df %>% dim() %>% print()
all_df %>% drop_na(c('array', 'imp')) %>% dim() %>% print()

[1] 36  4
[1] 35  4
